In [18]:
# import libraries
import os
from IPython.display import Markdown
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model

In [10]:
API_KEY = os.getenv('GOOGLE_API_KEY')
DOC_PATH = 'data/NIPS-2017-attention-is-all-you-need-Paper.pdf'
CHROMA_PATH = "db1" 

In [11]:
model = init_chat_model("gemini-2.0-flash-001", model_provider="google-genai")
ans = model.invoke("Hello, world!")
ans.content

'Hello there! How can I help you today?'

In [12]:
# ----- Data Indexing Process -----

# load your pdf doc
loader = PyPDFLoader(DOC_PATH)
pages = loader.load()

# split the doc into smaller chunks i.e. chunk_size=500
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(pages)

# get OpenAI Embedding model
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

# embed the chunks as vectors and load them into the database
db_chroma = Chroma.from_documents(chunks, embeddings, persist_directory=CHROMA_PATH)

In [19]:
# ----- Retrieval and Generation Process -----

# this is an example of a user question (query)
query = 'How many layers do authors use in encoder layer ?'

# retrieve context - top 5 most relevant (closests) chunks to the query vector 
# (by default Langchain is using cosine distance metric)
docs_chroma = db_chroma.similarity_search_with_score(query, k=5)

# generate an answer based on given user query and retrieved context information
context_text = "\n\n".join([doc.page_content for doc, _score in docs_chroma])

# you can use a prompt template
PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
Answer the question based on the above context: {question}.
Provide a detailed answer.
Don’t justify your answers.
Don’t give information not mentioned in the CONTEXT INFORMATION.
Do not say "according to the context" or "mentioned in the context" or similar.
"""

# load retrieved context and user query in the prompt template
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query)

# call LLM model to generate the answer based on the given context and query
model = init_chat_model("gemini-2.0-flash-001", model_provider="google-genai")
response_text = model.predict(prompt)
Markdown(response_text)

The encoder is composed of a stack of N = 6 identical layers. Each layer has two sub-layers.